In [194]:
import math
import traceback
import datetime

import numpy as np
import pandas as pd
import tushare as ts

ts.set_token('your token here')
# pro = ts.pro_api()
tu_token = 'fba0f69c02f5e4399b8dd63ae895ca69b7a55122a545c6f0f2c2bef3'
pro = ts.pro_api(tu_token)
start_date = '20100101'
# end_date='20210930'
end_date = '2028010'
first_money = 100000

df_res = {"交易码", "名称", "开始日期", "现金", "最新价格", "持股数", "总值"}
today = datetime.datetime.strptime("20211004", "%Y%m%d")


def get_item(ts_code):
    df = pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)[['trade_date', 'open', 'close']]
    df = df.sort_values(by='trade_date')
    start_price = df.iloc[0]['open']

    df['ma5'] = np.nan
    df['ma30'] = np.nan
    # for i in range(4, len(df)):
    #     df.loc[df.index[i], 'ma5'] = df['close'][i - 4:i + 1].mean()
    # for i in range(30, len(df)):
    #     df.loc[df.index[i], 'ma30'] = df['close'][i - 29:i + 1].mean()
    df['ma5'] = df['open'].rolling(5).mean()
    df['ma30'] = df['open'].rolling(30).mean()
    df = df.set_index('trade_date', drop=True)
    # df[['close','ma5','ma30']].plot()
    # plt.show()

    df = df.dropna()
    # # 循环计算
    # golden_cross = []
    # death_cross = []
    # for i in range(1, len(df)):
    #     if df['ma5'][i] >= df['ma30'][i] and df['ma5'][i - 1] < df['ma30'][i - 1]:
    #         golden_cross.append(df.index[i])
    #     if df['ma5'][i] <= df['ma30'][i] and df['ma5'][i - 1] > df['ma30'][i - 1]:
    #         death_cross.append(df.index[i])

    # 直接计算
    sr1 = df['ma5'] < df['ma30']
    sr2 = df['ma5'] > df['ma30']
    death_cross = (df[sr1 & sr2.shift(1)]).index
    golden_cross = (df[-(sr1 | sr2.shift(1))]).index
    # golden_cross
    sr_golden = pd.Series(1, index=golden_cross)
    sr_death = pd.Series(0, index=death_cross)

    #%%
    hold = 0
    sr_all = sr_golden.append(sr_death).sort_index()
    money = first_money
    for i in range(0, len(sr_all)):
        p = df['open'][sr_all.index[i]]
        # print("p = " , p)
        if 1 == sr_all.iloc[i]:
            # 金叉
            buy = (money // (100 * p))
            hold += buy * 100
            money -= buy * p * 100
            # print("%s : %d : %f"%(sr_all.index[i],buy,money))
        else:
            # 死叉
            money += hold * p
            hold = 0
    # print("%s : %f"%(sr_all.index[i],money))
    # 最后一天
    p = df['open'][-1]
    now_money = hold * p + money
    print("现金 ：%f,当前价格 %f ,持股 ：%d,总价 ： %f" % (money, p, hold, now_money))
    return {"现金": money,
            "初始价格": start_price,
            "最新价格": p,
            "持股数": hold,
            "总值": now_money}


df_res = pd.DataFrame(columns=("交易码", "名称", "开始日期", "初始价格", "现金", "最新价格", "持股数", "总值", "年化"))
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name,list_date',
                       start_date=start_date)
df_all = data.head(500000)
for index, row in df_all.iterrows():
    print("%d : %s" % (index, row['ts_code']))
    try:
        if str(start_date) > str(row['list_date']):
            tmp_start_date = start_date
        else:
            tmp_start_date = row['list_date']
        # 平安
        # ts_code = '601318.SH'
        item_res = get_item(row['ts_code'])
        item_res['交易码'] = row['ts_code']
        item_res['名称'] = row['name']
        item_res['开始日期'] = tmp_start_date
        year = float((today - (datetime.datetime.strptime(tmp_start_date, "%Y%m%d"))).days) / 365.0
        item_res['年化'] = (math.pow(item_res['总值'] / first_money, 1 / year) - 1) * 100
        # print(year)
        # print(item_res['年化'] )
        # print((time.strptime(tmp_start_date,"%Y%m%d")))
        # print(today)
        # print("%d" % (today - (datetime.datetime.strptime(tmp_start_date, "%Y%m%d"))).days)
        df_res = df_res.append(pd.DataFrame(item_res, index=[0]), ignore_index=True)
    except  Exception as e:
        # print(e)
        traceback.print_exc()
# print(tmp_start_date)
df_res.to_csv("../res.csv")
df_res


0 : 000001.SZ
现金 ：129781.000000,当前价格 18.090000 ,持股 ：0,总价 ： 129781.000000
1 : 000002.SZ
现金 ：82972.000000,当前价格 21.450000 ,持股 ：0,总价 ： 82972.000000
2 : 000004.SZ
现金 ：1265.000000,当前价格 18.570000 ,持股 ：5400,总价 ： 101543.000000
3 : 000005.SZ
现金 ：122327.000000,当前价格 2.070000 ,持股 ：0,总价 ： 122327.000000
4 : 000006.SZ
现金 ：61974.000000,当前价格 4.480000 ,持股 ：0,总价 ： 61974.000000
5 : 000007.SZ
现金 ：63931.000000,当前价格 4.040000 ,持股 ：0,总价 ： 63931.000000
6 : 000008.SZ
现金 ：194.000000,当前价格 2.250000 ,持股 ：6700,总价 ： 15269.000000
7 : 000009.SZ
现金 ：159373.000000,当前价格 18.350000 ,持股 ：0,总价 ： 159373.000000
8 : 000010.SZ
现金 ：25508.000000,当前价格 3.700000 ,持股 ：0,总价 ： 25508.000000
9 : 000011.SZ
现金 ：453.000000,当前价格 12.030000 ,持股 ：7800,总价 ： 94287.000000
10 : 000012.SZ
现金 ：246593.000000,当前价格 9.920000 ,持股 ：0,总价 ： 246593.000000
11 : 000014.SZ
现金 ：286.000000,当前价格 7.000000 ,持股 ：4900,总价 ： 34586.000000
12 : 000016.SZ
现金 ：338257.000000,当前价格 6.140000 ,持股 ：0,总价 ： 338257.000000
13 : 000017.SZ
现金 ：228136.000000,当前价格 3.480000 ,持股 ：0,总价 ： 228136.

Traceback (most recent call last):
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 100, in <module>
    item_res = get_item(row['ts_code'])
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 77, in get_item
    p = df['open'][-1]
  File "d:\code\python\money\venv\lib\site-packages\pandas\core\series.py", line 939, in __getitem__
    return self._values[key]
IndexError: index -1 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 100, in <module>
    item_res = get_item(row['ts_code'])
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 77, in get_item
    p = df['open'][-1]
  File "d:\code\python\money\venv\lib\site-packages\pandas\core\series.py", line 939, in __getitem__
    return self._values[key]
IndexError: index -1 is out of bounds for axis 0 with size 0


462 : 001215.SZ
463 : 001217.SZ


Traceback (most recent call last):
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 100, in <module>
    item_res = get_item(row['ts_code'])
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 77, in get_item
    p = df['open'][-1]
  File "d:\code\python\money\venv\lib\site-packages\pandas\core\series.py", line 939, in __getitem__
    return self._values[key]
IndexError: index -1 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 100, in <module>
    item_res = get_item(row['ts_code'])
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 77, in get_item
    p = df['open'][-1]
  File "d:\code\python\money\venv\lib\site-packages\pandas\core\series.py", line 939, in __getitem__
    return self._values[key]
IndexError: index -1 is out of bounds for axis 0 with size 0


464 : 001696.SZ
现金 ：135931.000000,当前价格 6.840000 ,持股 ：0,总价 ： 135931.000000
465 : 001872.SZ
现金 ：999.000000,当前价格 17.290000 ,持股 ：3000,总价 ： 52869.000000
466 : 001896.SZ
现金 ：262.000000,当前价格 8.400000 ,持股 ：33000,总价 ： 277462.000000
467 : 001914.SZ
现金 ：1325.000000,当前价格 14.630000 ,持股 ：31100,总价 ： 456318.000000
468 : 001965.SZ
现金 ：372.000000,当前价格 7.520000 ,持股 ：9100,总价 ： 68804.000000
469 : 001979.SZ
现金 ：1016.000000,当前价格 13.020000 ,持股 ：4400,总价 ： 58304.000000
470 : 002001.SZ
现金 ：117111.000000,当前价格 26.360000 ,持股 ：0,总价 ： 117111.000000
471 : 002002.SZ
现金 ：35470.000000,当前价格 5.520000 ,持股 ：0,总价 ： 35470.000000
472 : 002003.SZ
现金 ：455.000000,当前价格 8.710000 ,持股 ：5100,总价 ： 44876.000000
473 : 002004.SZ
现金 ：353.000000,当前价格 7.000000 ,持股 ：4800,总价 ： 33953.000000
474 : 002005.SZ
现金 ：39.000000,当前价格 1.930000 ,持股 ：12300,总价 ： 23778.000000
475 : 002006.SZ
现金 ：248.000000,当前价格 16.800000 ,持股 ：56900,总价 ： 956168.000000
476 : 002007.SZ
现金 ：27595.000000,当前价格 27.950000 ,持股 ：0,总价 ： 27595.000000
477 : 002008.SZ
现金 ：179354.000000,当前价

...

Traceback (most recent call last):
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 100, in <module>
    item_res = get_item(row['ts_code'])
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 77, in get_item
    p = df['open'][-1]
  File "d:\code\python\money\venv\lib\site-packages\pandas\core\series.py", line 939, in __getitem__
    return self._values[key]
IndexError: index -1 is out of bounds for axis 0 with size 0


4479 : 688777.SH
现金 ：87669.000000,当前价格 85.980000 ,持股 ：0,总价 ： 87669.000000
4480 : 688778.SH
现金 ：74818.000000,当前价格 108.880000 ,持股 ：0,总价 ： 74818.000000
4481 : 688779.SH
现金 ：100000.000000,当前价格 25.100000 ,持股 ：0,总价 ： 100000.000000
4482 : 688786.SH
4483 : 688787.SH


Traceback (most recent call last):
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 100, in <module>
    item_res = get_item(row['ts_code'])
  File "C:\Users\FlyPanda\AppData\Local\Temp/ipykernel_9756/2089955070.py", line 77, in get_item
    p = df['open'][-1]
  File "d:\code\python\money\venv\lib\site-packages\pandas\core\series.py", line 939, in __getitem__
    return self._values[key]
IndexError: index -1 is out of bounds for axis 0 with size 0


现金 ：100000.000000,当前价格 82.460000 ,持股 ：0,总价 ： 100000.000000
4484 : 688788.SH
现金 ：95766.000000,当前价格 131.210000 ,持股 ：0,总价 ： 95766.000000
4485 : 688789.SH
现金 ：99409.000000,当前价格 228.000000 ,持股 ：0,总价 ： 99409.000000
4486 : 688793.SH
现金 ：100000.000000,当前价格 90.020000 ,持股 ：0,总价 ： 100000.000000
4487 : 688798.SH
现金 ：100000.000000,当前价格 228.290000 ,持股 ：0,总价 ： 100000.000000
4488 : 688799.SH
现金 ：100000.000000,当前价格 38.210000 ,持股 ：0,总价 ： 100000.000000
4489 : 688800.SH
现金 ：88240.000000,当前价格 67.770000 ,持股 ：0,总价 ： 88240.000000
4490 : 688819.SH
现金 ：98440.000000,当前价格 43.040000 ,持股 ：0,总价 ： 98440.000000
4491 : 688981.SH
现金 ：83332.000000,当前价格 55.450000 ,持股 ：0,总价 ： 83332.000000
4492 : 689009.SH
现金 ：3463.000000,当前价格 80.100000 ,持股 ：1200,总价 ： 99583.000000


,交易码,名称,开始日期,初始价格,现金,最新价格,持股数,总值,年化
0,000001.SZ,平安银行,20100101,24.52,129781.0,18.09,0,129781.0,2.240557
1,000002.SZ,万科A,20100101,10.85,82972.0,21.45,0,82972.0,-1.574191
2,000004.SZ,国华网安,20100101,9.50,1265.0,18.57,5400.0,101543.0,0.130242
3,000005.SZ,ST星源,20100101,6.01,122327.0,2.07,0,122327.0,1.727788
4,000006.SZ,深振业A,20100101,11.33,61974.0,4.48,0,61974.0,-3.985389
...,...,...,...,...,...,...,...,...,...
4435,688799.SH,华纳药厂,20210713,61.20,100000.0,38.21,0,100000.0,0.000000
4436,688800.SH,瑞可达,20210722,68.00,88240.0,67.77,0,88240.0,-46.049093
4437,688819.SH,天能股份,20210118,80.00,98440.0,43.04,0,98440.0,-2.191416
4438,688981.SH,中芯国际,20200716,95.00,83332.0,55.45,0,83332.0,-13.891131
